In [1]:
# Install packages
!pip install -q langchain faiss-cpu sentence-transformers==2.2.2 InstructorEmbedding pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.9/393.9 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 566.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00


In [4]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.7 MB/s eta 0:00:00


In [5]:
from langchain.document_loaders import TextLoader
from pypdf import PdfReader
from langchain import HuggingFaceHub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferWindowMemory

In [6]:
# Load pdf documents
documents_1 = ''

reader = PdfReader('esaE10.pdf')
for page in reader.pages:
    documents_1 += page.extract_text()

documents_1[:300]

'Hand Soldering to  \nESA Specification ECSS -Q-ST-70-61  \n \nFact Sheet E 10 \nASTA Technology UK Ltd  \nReview date: 13/10/2023  This ESA- approved course will review the latest \nrevision of the ECSS -Q-ST-70-61 standard and is \nfollowed by an open book multiple choice test . The \nreview of the standar'

In [7]:
# Document Splitting
chunk_size = 200
chunk_overlap = 10

splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)
split_1 = splitter.split_text(documents_1)
split_1 = splitter.create_documents(split_1)

In [10]:
# Load embeddings instructor
instructor_embeddings = HuggingFaceInstructEmbeddings(
    model_name='hkunlp/instructor-xl'
)

# Implement embeddings
db = FAISS.from_documents(split_1, instructor_embeddings)

# Save db
db.save_local('vector store/esa')

load INSTRUCTOR_Transformer
max_seq_length  512


In [11]:
# Load db
loaded_db = FAISS.load_local(
    'vector store/esa', instructor_embeddings, allow_dangerous_deserialization=True
)

In [13]:
# Retrieve answer
question = 'Recertification'

search = loaded_db.similarity_search(question)
search

[Document(page_content='recertification course E 190 is available.   \n \nEnquiries are welcome – please use the websit e \ncontact form  for security.'),
 Document(page_content='Duration, schedule and fees 10 days – Please \nsee the current Course Calendar for dates and \nfees.  \n \nRecertification  \nTo renew certification for a further 2 years, a 4 - day'),
 Document(page_content='Award  \nIn accordance with ESA STR -258, an ESA approved, \nCategory 3 Certificate, valid for 2 years, will be awarded \non successful completion of the course for operators'),
 Document(page_content='and a Category 2 Certificate, valid for 2 years will be \nawarded on successful completion for inspectors . \n  \nExperience and pre -requisites  \n• Two years of industrial soldering experience')]

In [14]:
search_scores = loaded_db.similarity_search_with_score(question)
search_scores

[(Document(page_content='recertification course E 190 is available.   \n \nEnquiries are welcome – please use the websit e \ncontact form  for security.'),
  0.4496389),
 (Document(page_content='Duration, schedule and fees 10 days – Please \nsee the current Course Calendar for dates and \nfees.  \n \nRecertification  \nTo renew certification for a further 2 years, a 4 - day'),
  0.56722057),
 (Document(page_content='Award  \nIn accordance with ESA STR -258, an ESA approved, \nCategory 3 Certificate, valid for 2 years, will be awarded \non successful completion of the course for operators'),
  0.6070589),
 (Document(page_content='and a Category 2 Certificate, valid for 2 years will be \nawarded on successful completion for inspectors . \n  \nExperience and pre -requisites  \n• Two years of industrial soldering experience'),
  0.62421274)]

In [15]:
temperature = 1
max_length = 300
llm_model = 'tiiuae/falcon-7b-instruct'

# Load LLM
llm = HuggingFaceHub(
    repo_id=llm_model,
    model_kwargs={'temperature': temperature, 'max_length': max_length},
    huggingfacehub_api_token='your_api_token'
)

# Create the chatbot
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=loaded_db.as_retriever(),
    return_source_documents=True,
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  warn_deprecated(


In [20]:
# Ask a question
question = 'Renew certification'
response = qa({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
print(answer)
#explanation = response.get('source_documents', [])
#explanation

To renew certification for a further 2 years, a 4-day recertification course E 190 is available. Enquiries are welcome - please use the website contact form for security.
